<a href="https://colab.research.google.com/github/luck058/playingWithML/blob/mlflow/PlayingWithML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## https://www.youtube.com/watch?v=c36lUUr864M
Pt. 13

## Imports/config

### Imports

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
import math

### Mount google drive

### Set up mlflow

In [4]:
experiment_name = "/Users/unknownapple89@gmail.com/PlayingWithML" #@param {type:"string"}

!pip install mlflow -q
!databricks configure --host https://community.cloud.databricks.com/

import mlflow
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment(experiment_name)

Username: Aborted!
^C


PydanticUserError: ignored

### Device configuration

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyperparameters

In [ ]:
input_size = 5
output_size = 1
hidden_size = 2
num_epochs = 10
batch_size = 4
learning_rate = 0.001


## Data

### Import data

In [ ]:
class LinearDataset(Dataset):
  def __init__(self, train):
    self.x = []
    self.y = []
    self.x_test = []
    self.y_test = []
    self.train = train

    # Includes test and training data
    num_samples = 50

    full_graph = torch.tensor([2*(i) for i in range(num_samples + input_size + output_size)], dtype=torch.float32)

    for i in range(len(full_graph) - input_size - output_size):
      if i%10 != 0:
        self.x.append(full_graph[i : i+input_size])
        self.y.append(full_graph[i+input_size : i+input_size+output_size])
      else:
        self.x_test.append(full_graph[i : i+input_size])
        self.y_test.append(full_graph[i+input_size : i+input_size+output_size])

  def __getitem__(self, index):
    if self.train:
      return self.x[index], self.y[index]
    else:
      return self.x_test[index], self.y_test[index]

  def __len__(self):
    if self.train:
      return len(self.x)
    else:
      return len(self.x_test)


training_dataset = LinearDataset(train=True)
testing_dataset = LinearDataset(train=False)

train_loader = DataLoader(dataset=training_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=testing_dataset, batch_size=batch_size, shuffle=False)

total_samples = len(training_dataset)

print("Length of training data:", len(training_dataset))
print("Length of test data:", len(testing_dataset))

### Visualise data

In [ ]:
dataiter = iter(train_loader)
(inputs, labels) = next(dataiter)
for i in range(batch_size):
  plt.subplot(2,math.ceil(batch_size/2),i+1)
  print(inputs[i])
  print(labels[i])
  plt.plot(inputs[i], 'b+')
  plt.plot(input_size, labels[i], 'r+')

plt.show()

## Other function definitions

In [ ]:
def get_test_loss(model, test_loader):
    with torch.no_grad():
      for i, (test_inputs, test_labels) in enumerate(test_loader):
          test_outputs = model(test_inputs)
          test_loss = criterion(test_outputs, test_labels)
      return test_loss

## Model

### Model definition

In [ ]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out

### Model, loss and optimiser

In [ ]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)

loss_history = []
test_history = []

print(model.parameters)

print("------------------")

print(model.parameters()["l1"])

print("------------------")


for p in model.parameters():
  print("\n###################\n")
  print(p)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training loop

In [ ]:
# Train the model
n_total_steps = len(train_loader)
test_history.append(get_test_loss(model, test_loader))

for epoch in range(num_epochs):
    loss_history.append([])
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history[-1].append(loss)

        if (i+1) % (n_total_steps//1) == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

    # Find loss on test data
    test_history.append(get_test_loss(model, test_loader))

## Testing

### Loss graph

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
first_epoch_shown = 2
last_epoch_shown = 10

fig, ax = plt.subplots()

last_plotted_point = [loss_history[0][0]]

with torch.no_grad():
  # Plot the data from the given epoch. Also plots the last datapoint from the previous epoch so the lines join nicely
  epoch_length = len(loss_history[0])
  for epoch, loss_hist_epoch in enumerate(loss_history):
    if epoch_length != len(loss_hist_epoch):
      print("Epoch length changes")

    if epoch >= first_epoch_shown and epoch <= min(last_epoch_shown, len(loss_history)):
      start_epoch_step = epoch * epoch_length
      start_next_epoch_step = (epoch+1) * epoch_length
      # Plot loss graph of training data
      ax.plot(list(range(start_epoch_step-1, start_next_epoch_step)), last_plotted_point + loss_hist_epoch)

    last_plotted_point = [loss_hist_epoch[-1]]

  ax.plot(list(range(first_epoch_shown*len(), last_epoch_shown)), test_history)

  ax.set_xlabel("Steps")
  ax.set_ylabel("Loss")
  secax = ax.secondary_xaxis('top', functions=(lambda x: x/len(loss_hist_epoch), lambda x: x*len(loss_hist_epoch)))
  secax.set_xlabel("Epochs")
  plt.show()

### Final model prediction on test data

In [ ]:
with torch.no_grad():
    outputs = model(inputs)

## Test functions

In [ ]:
a = nn.ReLU(2.5)
print(a(torch.tensor([-1, 1, 3])))

b = NeuralNet(1, 2, 3)
print(b)
for p in b.parameters():
  print(p)
print("\n")
print(b.relu(torch.tensor([-1, 2, 3])))

In [ ]:
mlflow.log_param("num_dimensions", 8)
!mlflow ui